In [1]:
%set_env SCIPY_USE_PROPACK=1

import config

import numpy as np
import ctypes
import sys
from numpy.linalg import lstsq
from numba import jit, njit
import matplotlib.pyplot as plt
import pickle

from scipy.sparse import *
from scipy.sparse.linalg import lsmr, lsqr, eigs

import importlib
import FORBILD

importlib.reload(FORBILD)
importlib.reload(config)

from FORBILD import forbild_sinogram, forbild_sinogram_noquad, discrete_phantom
from config import *

np.set_printoptions(threshold=sys.maxsize, precision=3, linewidth=480)


def forward_with_matrix(img, mat):
    img = img.reshape(nx*ny, 1)
    mat = mat.reshape(nnp*nu, nx*ny)

    proj = mat @ img

    return proj.reshape(nnp, nu)

def backward_from_projection(sino, mat):
    sino = sino.reshape(nnp*nu, 1)
    mat = mat.reshape(nnp*nu, nx*ny)
    
    img, residual, _, _ = lstsq(mat, sino, rcond=None)

    print("Residual: ", residual)

    return img.reshape(nx, ny)

def backward_from_projection_hex(sino, mat):
    sino = sino.reshape(nnp*nu, 1)
    mat = mat.reshape((nnp*nu, -1))
    
    img, residual, rank, _ = lstsq(mat, sino, rcond=None)

    print("Residual: ", residual, "Rank: ", rank)

    return img

def hex_mat_presolve(mat):
    mat = mat.reshape(nnp*nu, -1)
    mat1 = np.zeros((nnp*nu, nl*nc-np.floor(nl/2).astype(int)), dtype=np.float64)
    pos = pos1 = 0

    for i in range(nl):
        if i%2 == 0:
            mat1[:, pos1:pos1+nc] = mat[:, pos:pos+nc]
            pos1 += nc
            pos += nc
        else:
            mat1[:, pos1:pos1+nc-1] = mat[:, pos:pos+nc-1]
            pos1 += nc-1
            pos += nc
    
    return mat1

def post_process_hex(img):
    img1 = np.zeros((nl, nc), dtype=np.float64)
    pos = 0

    for i in range(nl):
        if i%2 == 0:
            img1[i] = img[pos:pos+nc].ravel()
            pos += nc
        else:
            img1[i, :-1] = img[pos:pos+nc-1].ravel()
            pos += nc-1
    
    return img1

env: SCIPY_USE_PROPACK=1


In [2]:
A_B0_sparse = pickle.load(open("./matrixes/A_B0_sparse_" + str(nx) + '_' + str(ny) + "_" + str(nnp) + "_"+ str(nu) + ".pkl", 'rb'))
A_H0_sparse = pickle.load(open("./matrixes/A_H0_sparse_" + str(H0_nl) + '_' + str(H0_nc) + "_" + str(nnp) + "_"+ str(nu) + ".pkl", 'rb'))

In [3]:
from scipy.sparse.linalg import svds

In [4]:
B0_sm = svds(A_B0_sparse, k=1, which='SM', return_singular_vectors=False, solver='propack')
print(B0_sm)
H0_sm = svds(A_H0_sparse, k=1, which='SM', return_singular_vectors=False, solver='propack')
print(H0_sm)

[0.]
[0.]
